<a href="https://colab.research.google.com/github/jamiemcgarry/Using_Frame_Semantic_Transfomer_on_historical_texts/blob/main/Chi_sqaure_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First start this notebook by importing at least two of the pkl files in the repository. In my case I'm importing the Final_Burke.pkl and Final_Paine,pkl from my google drive, however if your not using google colab then you will need to import it from yout desktop  

In [54]:
!pip install frame-semantic-transformer

In [55]:
!pip install scipy

In [56]:
import numpy as np
import pandas as pd

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import pickle

file_path = '/content/drive/My Drive/Final_Burke.pkl'

with open(file_path, 'rb') as file:
    serialized_result = file.read()

Final_Burke = pickle.loads(serialized_result)

In [59]:
len(Final_Burke)

3379

In [60]:
Burke_frame_list = []

for sentence in Final_Burke:
  for frame in sentence.frames:
    Burke_frame_list.append(frame.name)
Burke_size = len(Burke_frame_list)

In [61]:
len(Burke_frame_list)

18753

In [62]:
from collections import Counter

Burke_freq_dic = dict(Counter(Burke_frame_list))


In [63]:
Burke_df = pd.DataFrame(index = Burke_freq_dic.keys(), data = Burke_freq_dic.values(), columns = ['frequency'])

total_frequency = Burke_df['frequency'].sum()

#Burke_df

In the preceding cells I import The Final_Burke.pkl file which contains the data for all of Burke's text, and collect all the frames within. Then I count the frequencies and put it in a dictionary which is then converted into a pandas dataframe

In [64]:
import pickle

file_path = '/content/drive/My Drive/Final_Paine.pkl'

with open(file_path, 'rb') as file:
    serialized_result = file.read()

Final_Paine = pickle.loads(serialized_result)

In [65]:
len(Final_Paine)

3119

In [66]:
Paine_frame_list = []

for sentence in Final_Paine:
  for frame in sentence.frames:
    Paine_frame_list.append(frame.name)


Paine_size = len(Paine_frame_list)

In [67]:
Paine_size

18977

In [68]:
Paine_freq_dic = dict(Counter(Paine_frame_list))

In [69]:
Paine_df = pd.DataFrame(index = Paine_freq_dic.keys(), data = Paine_freq_dic.values(), columns = ['frequency'])

#total_frequency = Paine_df['frequency'].sum()

#Paine_df

I do the same with Thomas Paine data here

In [70]:
all_frames = Burke_df.index.tolist()

for i in Paine_df.index:
  if i not in all_frames:
    all_frames.append(i)
  else:
    continue

len(all_frames)

750

In [71]:
titles = ['Burke', 'Paine',]

comparative_df = pd.DataFrame(index = all_frames, columns = titles)

for i in comparative_df.index:
  if i in Burke_df.index:
    comparative_df.loc[i, 'Burke'] = Burke_df.loc[i, 'frequency']
  else:
    comparative_df.loc[i, 'Burke'] = 0
  if i in Paine_df.index:
    comparative_df.loc[i, 'Paine'] = Paine_df.loc[i, 'frequency']
  else:
    comparative_df.loc[i, 'Paine'] = 0

In [72]:
comparative_df['row totals'] = comparative_df.sum(axis=1)

comparative_df.loc['column totals'] = comparative_df.sum(axis=0)



In [73]:
for i in comparative_df.index:
  if comparative_df.loc[i, 'row totals'] <= 10:
    comparative_df.drop(i, inplace=True)
  else:
    continue

comparative_df

,Burke,Paine,row totals
Text,71,213,284
Sending,24,28,52
Desiring,199,100,299
Opinion,233,99,332
Supply,71,30,101
...,...,...,...
Sign_agreement,5,9,14
Secrecy_status,8,7,15
Arrest,2,10,12
Importing,1,10,11


Here I then combine the frequencies of both Burke and Paine's freqeuncies and remove that frames that have a total row total below 10

In [74]:
observed_df = comparative_df.iloc[0:396, 0:2]

In [75]:
import scipy.stats as stats

chi_results = stats.chi2_contingency(observed = observed_df)


This code then runs a chi-sqaure test of independence on the dataframe that has both Burke and Paines frequencies for each frame. It gives back the chi-value, the degrees of freedom, p-value and an array of expected frequencies.

In [76]:
chi2_stat = chi_results[0]
#chi2_stat

In [77]:
r = observed_df.shape[0]
n = observed_df.sum().sum()
k = observed_df.shape[1]
cramers_v = (chi2_stat / (n * min(k - 1, r - 1))) ** 0.5
cramers_v

0.2398923634326325

Here I run a Cramers v test to test gather the effect size of the chi-sqaured test of independence

In [78]:
expected_freq = chi_results[3]

In [79]:
observed_freq_list = []

for i in observed_df.index:
  array = [observed_df.loc[i, 'Burke'], observed_df.loc[i, 'Paine']]
  observed_freq_list.append(array)

observed_freq_array = np.array(observed_freq_list)


In [80]:
from scipy.stats import chi2

df = observed_freq_array.shape[1] - 1

p_values_final = []

for i in range(len(observed_freq_array)):
    p_values = []
    for e in range(observed_freq_array.shape[1]):
        chi_square = np.sum(np.square(observed_freq_array[i, e] - expected_freq[i, e]) / expected_freq[i, e])
        p_value = 1 - chi2.cdf(chi_square, df)
        p_values.append(p_value)
    p_values_final.append(p_values)

p_values_final = np.array(p_values_final)


array([[3.71056774e-09, 4.73495387e-09],
       [7.20020399e-01, 7.21857617e-01],
       [3.37315993e-05, 3.81683532e-05],
       [1.11447071e-07, 1.35936656e-07],
       [3.24307746e-03, 3.46065055e-03],
       [2.50764765e-03, 2.68457568e-03],
       [2.05637457e-02, 2.14458426e-02],
       [8.13914316e-01, 8.15165385e-01],
       [9.31171660e-03, 9.80626820e-03],
       [3.99089911e-01, 4.02326337e-01],
       [1.45642844e-02, 1.52533307e-02],
       [1.75335831e-04, 1.94201961e-04],
       [3.64132477e-01, 3.67426291e-01],
       [2.77543709e-01, 2.80847020e-01],
       [5.70788616e-01, 5.73428920e-01],
       [4.02012993e-02, 4.15873420e-02],
       [6.60356504e-02, 6.79112957e-02],
       [1.93270069e-01, 1.96337086e-01],
       [6.38026385e-01, 6.40329156e-01],
       [0.00000000e+00, 0.00000000e+00],
       [4.25938462e-01, 4.29113577e-01],
       [6.92790976e-02, 7.12067752e-02],
       [4.06156647e-01, 4.09378314e-01],
       [8.47451879e-01, 8.48483736e-01],
       [5.177896

I then use the array of expected frequencies to run a chi-sqaure test on every freqenucy found in the obersved_df. This is done using nested for loop which finds the position of both the relevant values in the observed and expected arrays to run the calculation and create and array of p-values

In [81]:
Burke_list = []
Paine_list = []

for i in p_values_final:
  Burke_list.append(i[0])
  Paine_list.append(i[1])

len(Paine_list)

396

In [85]:
observed_df['p values Burke'] = Burke_list
observed_df['p values Paine'] = Paine_list
pd.options.display.float_format = '{:.10f}'.format
observed_df

,Burke,Paine,p values Burke,p values Paine
Text,71,213,0.0000000037,0.0000000047
Sending,24,28,0.7200203989,0.7218576168
Desiring,199,100,0.0000337316,0.0000381684
Opinion,233,99,0.0000001114,0.0000001359
Supply,71,30,0.0032430775,0.0034606505
...,...,...,...,...
Conditional_occurrence,7,12,0.4279772604,0.4311471533
Sign_agreement,5,9,0.4591190851,0.4621999718
Secrecy_status,8,7,0.8398494792,0.8409313870
Arrest,2,10,0.1048581322,0.1072583036


In [87]:
Burke_Paine_significance = observed_df[observed_df['p values Paine'] < 0.05]
Burke_Paine_significance

,Burke,Paine,p values Burke,p values Paine
Text,71,213,0.0000000037,0.0000000047
Desiring,199,100,0.0000337316,0.0000381684
Opinion,233,99,0.0000001114,0.0000001359
Supply,71,30,0.0032430775,0.0034606505
Aggregate,197,123,0.0025076476,0.0026845757
...,...,...,...,...
Medical_intervention,12,1,0.0290870615,0.0302072919
Commerce_pay,7,27,0.0161589857,0.0169019006
People_by_origin,7,37,0.0014895281,0.0016051792
Building_subparts,8,30,0.0123411969,0.0129514541


By putting the p-values of every frame in the original 'observed_df', it allows me to create the dataframe in the cell above which contains only those difference which are statistically significant